In [62]:
from utils.df_handle import *

In [63]:
URL_saleinput ='https://docs.google.com/spreadsheets/d/e/2PACX-1vSYWdHL8hh3RoQTe4dAaidvJ83sbrKiZm9gseWgNru2uWRtfpKzYY87ix9rjl5AK5F7Fa3VGpGA4QgQ/pub?gid=1035878545&single=true&output=csv'

In [64]:
df = pd.read_csv(URL_saleinput,header = 0)

In [66]:
report_links=df.Link

In [67]:
index =0
df2 = pd.DataFrame()
for link in report_links:
    print(index, link)
    index += 1
    df1 = pd.read_csv(link)
    df1.columns = cleancols(df1)
    df1.columns = lower_col(df1)
    df2 =pd.concat([df2, df1],ignore_index=True).drop_duplicates()
else:
    print('Đã hết số')




0 https://docs.google.com/spreadsheets/d/e/2PACX-1vS7M8qr9Iz5eU1LzIe5e-_4ng7LfUibZo4pEk9O5Kn1WxbsGXAWU0dEn37j0oZUE1zbIaGqfDH_H4F2/pub?gid=0&single=true&output=csv
1 https://docs.google.com/spreadsheets/d/e/2PACX-1vR1t6SUVfRr25uwnUKB1D-s0OcJTC7oGeOmwBwfqWcwXKstw640TG3Ma1JTLbSqGVnLQ74OmQvwET3P/pub?gid=0&single=true&output=csv
2 https://docs.google.com/spreadsheets/d/e/2PACX-1vSos2H4ERb-w7TI-YyITz1Iklmo89rs0k202NCXu1M-77uIC67-ul_V17tzrt9tsSELpDfbfgd11XqJ/pub?gid=0&single=true&output=csv
3 https://docs.google.com/spreadsheets/d/e/2PACX-1vR4-y9ok05alh2489hMrxT7ZJydBA6RPZ-owQNph75qumtpj5hjlqlPxlPVZEygtbSt3069O5zuA03G/pub?gid=0&single=true&output=csv
4 https://docs.google.com/spreadsheets/d/e/2PACX-1vT2qOAPQTLCja_YRZVh84OXmOS3A891f3Yv_6sirnPoAHvzmUK3yzz0g8cBwVeQh2KVUtPt0BgyM8cg/pub?gid=0&single=true&output=csv
5 https://docs.google.com/spreadsheets/d/e/2PACX-1vQmoou01dvq3VKpihmb2PkVBqHjuIcUJ8ZqAF_LiUOMN6ATrbPHps7Y1RkFazZV6qrtGpk8NV8tNEWS/pub?gid=0&single=true&output=csv
6 https://docs.google.

In [70]:
# Dùng để mapping tuần sang ngày
data_convert = pd.read_csv("convert_date.csv", dayfirst=True, parse_dates=['ngay'])
reference = pd.DataFrame(data_convert)

In [71]:
df_unpivoted = df2.melt(id_vars=['makhcu','manhanvien','tenkhachhang','tennhanvien','giaiphap'], 
                       var_name='tuan', value_name='thongtinthanhtoan')
df3 = pd.DataFrame(df_unpivoted)

In [72]:
df4=df3.merge(reference, on='tuan', how='left')
df5=pd.DataFrame(df4)

In [74]:
df5.rename({'ngay':'thoigiangoi','manhanvien':'manv'}
          ,axis='columns',inplace =True)
df5['updated_at'] = datetime.now()

In [75]:
df5['thoigiangoi'] = pd.to_datetime(df5['thoigiangoi'], format='%Y%m%d %H%M%S')


In [76]:
df6=df5[['manv','thongtinthanhtoan','giaiphap','thoigiangoi','makhcu','updated_at']]

In [77]:
df7=df6.dropna(subset=['manv','thongtinthanhtoan'])

In [79]:
def upsert_table():
	df = df7
	pk = ['manv','makhcu','thongtinthanhtoan']
	execute_values_upsert(df, "d_tttt_ins",pk)

In [80]:
def main():  
    upsert_table()

In [81]:
if __name__ == "__main__":
    main()

INSERT INTO d_tttt_ins(manv,thongtinthanhtoan,giaiphap,thoigiangoi,makhcu,updated_at) VALUES %s ON CONFLICT (manv, makhcu, thongtinthanhtoan) DO UPDATE SET giaiphap = excluded.giaiphap, thoigiangoi = excluded.thoigiangoi, updated_at = excluded.updated_at
Data inserted using execute_values() successfully..
